# Metric RAG computation

This computes the Euclidean distance for each establishment from a base establishment of a consistent type (Academies, Maintained schools, SEN). Each establishment is given a weight against every other establishment and the top 60 for each establishment creates the comparator set for that establishment. This is repeated for all establishments in the establishment types set and then finally across all establishments across all establishment types.

In [1]:
import numpy as np
import pandas as pd
import calculations as calcs
import glob 
import os

# Create and clean directory
from pathlib import Path
Path("output/metric-rag").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/metric-rag/*")
for f in files:
    os.remove(f)

## Parameter lists

The following setup and parameter map for each establishment type calculation. Vectors with `_parameter_list` suffix represent a column read from one of the pre-processed datasets. Vectors with `_parameter_weightings` suffix represent the corresponding weight to be applied to Euclidean distant component; 

In [2]:
sen_breakdown_parameter_list = ['EHC_Primary_need_spld','EHC_Primary_need_mld','EHC_Primary_need_sld',
                      'EHC_Primary_need_pmld','EHC_Primary_need_semh','EHC_Primary_need_slcn','EHC_Primary_need_hi','EHC_Primary_need_vi','EHC_Primary_need_msi','EHC_Primary_need_pd','EHC_Primary_need_asd','EHC_Primary_need_oth']
        
school_parameter_list = ['NumberOfPupils','Percentage Free school meals','Percentage SEN']
school_parameter_weightings = [0.5,0.4,0.1]


special_school_parameter_list = ['Number of pupils','Percentage Free school meals',
                                 'EHC_Primary_need_mld','EHC_Primary_need_sld','EHC_Primary_need_pmld',
                                 'EHC_Primary_need_semh','EHC_Primary_need_slcn','EHC_Primary_need_hi',
                                 'EHC_Primary_need_vi','EHC_Primary_need_msi','EHC_Primary_need_pd','EHC_Primary_need_asd','EHC_Primary_need_oth']
special_school_parameter_weightings = [0.6,0.4,1,1,1,1,1,1,1,1,1,1,1,1]

school_area_parameter_list = ['Total Internal Floor Area','Age Average Score']
school_area_parameter_weightings = [0.8,0.2]

## Academy pupil mix

In [3]:
academy_data = pd.read_csv("output/pre-processing/academies.csv")
academy_data['Boarders (name)'] = academy_data['Boarders (name)'].map(lambda x : 'Not Boarding' if x == 'Unknown' else x)
academy_data['NumberOfPupils'] = academy_data['NumberOfPupils'].fillna(academy_data['NumberOfPupils'].median())
academy_data['Percentage Free school meals'] = academy_data['Percentage Free school meals'].fillna(academy_data['Percentage Free school meals'].median())
academy_data['Percentage SEN'] = academy_data['Percentage SEN'].fillna(academy_data['Percentage SEN'].median())

grouped_academy_data = academy_data.groupby(['SchoolPhaseType','Boarders (name)']).agg(list)
grouped_academy_data = (grouped_academy_data.join(grouped_academy_data['NumberOfPupils'].map(calcs.compute_range).rename('Pupil_Range'))
                    .join(grouped_academy_data['Percentage Free school meals'].map(calcs.compute_range).rename('FSM_Range'))
                    .join(grouped_academy_data['Percentage SEN'].map(calcs.compute_range).rename('SEN_Range')))

special_excluded = grouped_academy_data[grouped_academy_data.index.get_level_values('SchoolPhaseType') != 'Special']
special_excluded

URN  \
SchoolPhaseType              Boarders (name)                                                      
Alternative Provision        Not Boarding     [148144, 145704, 141109, 142569, 144031, 13966...   
Post-16                      Boarding                                          [140971, 148791]   
                             Not Boarding     [144463, 144886, 144740, 143704, 145749, 13989...   
Primary                      Boarding                                          [147676, 147857]   
                             Not Boarding     [148853, 144542, 144551, 148854, 136354, 14021...   
Secondary                    Boarding         [148221, 139655, 138289, 136177, 139971, 13915...   
                             Not Boarding     [136730, 136844, 137377, 137083, 144809, 13703...   
University technical college Not Boarding     [146375, 141111, 139268, 145155, 146303, 14593...   

                                                                    Company Registration Number  \
SchoolPhaseType              Boarders (name)                                                      
Alternative Provision        Not Boarding     [11788031, 11788031, 8641815, 6207590, 8299181...   
Post-16                      Boarding                                        [8515877, 9320523]   
                             Not Boarding     [10578239, 8506178, 9802491, 2303464, 11200244...   
Primary                      Boarding                                       [10749662, 7466353]   
                             Not Boarding     [10817580, 10817580, 10817580, 10817580, 73187...   
Secondary                    Boarding         [7535642, 8098956, 8082185, 7209122, 7930340, ...   
                             Not Boarding     [7595434, 7595434, 7740516, 7705552, 7705552, ...   
University technical college Not Boarding     [8707909, 8707909, 8707909, 8707909, 5670663, ...   

                                                                             Incorporation Date  \
SchoolPhaseType              Boarders (name)                                                      
Alternative Provision        Not Boarding     [2019-01-25 00:00:00.0000000, 2019-01-25 00:00...   
Post-16                      Boarding         [2013-05-03 00:00:00.0000000, 2014-11-20 00:00...   
                             Not Boarding     [2017-01-23 00:00:00.0000000, 2013-04-26 00:00...   
Primary                      Boarding         [2017-05-02 00:00:00.0000000, 2010-12-10 00:00...   
                             Not Boarding     [2017-06-14 00:00:00.0000000, 2017-06-14 00:00...   
Secondary                    Boarding         [2011-02-18 00:00:00.0000000, 2012-06-08 00:00...   
                             Not Boarding     [2011-04-07 00:00:00.0000000, 2011-04-07 00:00...   
University technical college Not Boarding     [2013-09-26 00:00:00.0000000, 2013-09-26 00:00...   

                                                                             Academy Trust UPIN  \
SchoolPhaseType              Boarders (name)                                                      
Alternative Provision        Not Boarding     [151922, 151922, 137840, 135065, 137417, 13688...   
Post-16                      Boarding                                          [137977, 135984]   
                             Not Boarding     [139638, 137855, 140016, 134876, 141412, 13725...   
Primary                      Boarding                                          [139827, 135711]   
                             Not Boarding     [139821, 139821, 139821, 139821, 135399, 13539...   
Secondary                    Boarding         [135944, 137050, 137026, 135369, 136804, 13490...   
                             Not Boarding     [134890, 134890, 136150, 135666, 135666, 13566...   
University technical college Not Boarding     [137921, 137921, 137921, 137921, 135047, 13663...   

                                                                                  Academy UKPRN  \
SchoolPhaseType              

In [15]:
sets = pd.DataFrame()
for (idx, row) in special_excluded.iterrows():
    distances = calcs.non_special_distance_calc(row['URN'], row['NumberOfPupils'], row['Percentage Free school meals'], row['Percentage SEN'], row['Pupil_Range'], row['FSM_Range'], row['SEN_Range'])
    df = (pd.DataFrame(distances, columns=['URN', 'Candidates', 'Distance'])
                .sort_values(by=['URN', 'Distance'], ascending=[False,True])
                .groupby('URN')
                .agg(list)
                .apply(lambda d: {'Candidates': d['Candidates'][:60], 'Distance': d['Distance'][:60]}, axis=1, result_type='expand')
            )
    df.reset_index(inplace=True)
    df['SchoolPhaseType'] = idx[0]
    df['Boarders (name)'] = idx[1]
    df.set_index(['SchoolPhaseType','Boarders (name)','URN'], inplace=True)
    sets = pd.concat([sets, df])
    

sets

Candidates  \
SchoolPhaseType              Boarders (name) URN                                                           
Alternative Provision        Not Boarding    138264.0  [138264.0, 143761.0, 139925.0, 147855.0, 14747...   
                                             138277.0  [138277.0, 142259.0, 142793.0, 139509.0, 13950...   
                                             138379.0  [138379.0, 142835.0, 144023.0, 140627.0, 14789...   
                                             138775.0  [138775.0, 141739.0, 139413.0, 140649.0, 13982...   
                                             139114.0  [139114.0, 142702.0, 139665.0, 148588.0, 14322...   
...                                                                                                  ...   
University technical college Not Boarding    146560.0  [146560.0, 142900.0, 145900.0, 140160.0, 14260...   
                                             146648.0  [146648.0, 147685.0, 142899.0, 145900.0, 14289...   
                                             147685.0  [147685.0, 142890.0, 142899.0, 142605.0, 14026...   
                                             147838.0  [147838.0, 144766.0, 140987.0, 139234.0, 14096...   
                                             148932.0  [148932.0, 148932.0, 148932.0, 142130.0, 14181...   

                                                                                                Distance  
SchoolPhaseType              Boarders (name) URN                                                          
Alternative Provision        Not Boarding    138264.0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0082868920624...  
                                             138277.0  [0.0, 0.0, 0.006734350297014739, 0.16934953291...  
                                             138379.0  [0.0, 0.15735913879522825, 0.17037527409942446...  
                                             138775.0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
                                             139114.0  [0.0, 0.03237893551141869, 0.06203413424916427...  
...                                                                                                  ...  
University technical college Not Boarding    146560.0  [0.0, 0.11567928444989008, 0.22854898114198458...  
                                             146648.0  [0.0, 0.07482560347671036, 0.10047620295547079...  
                                             147685.0  [0.0, 0.0902955379139546, 0.17281887853211256,...  
                                             147838.0  [0.0, 0.10695042009140009, 0.1425305213068696,...  
                                             148932.0  [0.0, 0.0, 0.0, 0.09467483204176241, 0.1110228...  

[9364 rows x 2 columns]

## Maintained school pupil mix

## All pupil mix

## Academy area mix

## Maintained school area mix

## All area mix